This notebook demonstrates the fundamental ways to interact with foundation models via Amazon Bedrock using Python.

In [ ]:
import boto3
import json
from botocore.exceptions import ClientError
import time
from IPython.display import Markdown, display


Run the following cell only if running in Colab

In [ ]:
from google.colab import userdata

os.environ['AWS_ACCESS_KEY_ID'] = userdata.get('aws_access_key_id')
os.environ['AWS_SECRET_ACCESS_KEY'] = userdata.get('aws_secret_access_key')

In [ ]:

# Initialize the Bedrock client
# Make sure you have configured your AWS credentials
# Change the region if needed
bedrock_runtime = boto3.client(
    service_name='bedrock-runtime',
    region_name='us-east-1'
)

print("Bedrock client initialized successfully!")

Let's define some helper functions to work with Bedrock.

In [ ]:
def invoke_model(model_id, prompt, max_tokens=512, temperature=0.7):
    """
    Invokes a foundation model through Amazon Bedrock.
    
    Parameters:
    - model_id: The identifier for the model (e.g., "anthropic.claude-3-haiku-20240307")
    - prompt: The prompt text to send to the model
    - max_tokens: Maximum number of tokens to generate
    - temperature: Controls randomness (0.0 = deterministic, 1.0 = creative)
    
    Returns:
    - The model's response
    """
    try:
        # Create the appropriate request body based on model provider
        if "anthropic.claude" in model_id:
            # Anthropic Claude models
            request_body = {
                "anthropic_version": "bedrock-2023-05-31",
                "max_tokens": max_tokens,
                "temperature": temperature,
                "messages": [
                    {
                        "role": "user",
                        "content": prompt
                    }
                ]
            }
        elif "meta.llama" in model_id:
            # Meta Llama models
            request_body = {
                "prompt": prompt,
                "max_gen_len": max_tokens,
                "temperature": temperature
            }
        elif "amazon.titan" in model_id:
            # Amazon Titan models
            request_body = {
                "inputText": prompt,
                "textGenerationConfig": {
                    "maxTokenCount": max_tokens,
                    "temperature": temperature,
                    "topP": 0.9
                }
            }
        elif "ai21.j2" in model_id:
            # AI21 Jurassic models
            request_body = {
                "prompt": prompt,
                "maxTokens": max_tokens,
                "temperature": temperature
            }
        else:
            print(f"Model {model_id} not explicitly supported in this demo.")
            return None
            
        # Convert the request body to JSON string
        body = json.dumps(request_body)
        
        # Invoke the model
        print(f"Sending request to {model_id}...")
        response = bedrock_runtime.invoke_model(
            modelId=model_id,
            body=body
        )
        
        # Parse and return the response
        response_body = json.loads(response.get('body').read())
        
        # Extract the generated text based on model provider
        if "anthropic.claude" in model_id:
            return response_body["content"][0]["text"]
        elif "meta.llama" in model_id:
            return response_body["generation"]
        elif "amazon.titan" in model_id:
            return response_body["results"][0]["outputText"]
        elif "ai21.j2" in model_id:
            return response_body["completions"][0]["data"]["text"]
        else:
            return response_body
            
    except ClientError as e:
        print(f"Error invoking model: {e}")
        return None

## 1. Basic Model Invocation

Let's start with a simple example of invoking a model:

In [ ]:
# Define our model and prompt
model_id = "anthropic.claude-3-haiku-20240307"  # Change to any Bedrock model you have access to
prompt = "Explain the concept of foundation models in AI to a software developer."

# Call the model
response = invoke_model(model_id, prompt)

# Display the response in markdown for better readability
display(Markdown(f"## Response from {model_id}"))
display(Markdown(response))

## 2. Comparing Different Models

Now let's see how to compare responses from different models:

In [ ]:
def compare_models(prompt, models):
    """
    Compare responses from multiple models for the same prompt
    
    Parameters:
    - prompt: The prompt to send to all models
    - models: List of model IDs to compare
    
    Returns:
    - Dictionary mapping model IDs to their responses
    """
    results = {}
    for model_id in models:
        print(f"Querying {model_id}...")
        response = invoke_model(model_id, prompt)
        results[model_id] = response
        # Add a small delay to avoid rate limiting
        time.sleep(1)
    return results

# Define our prompt and models to compare
comparison_prompt = "What are the key features of Amazon Bedrock?"
models_to_compare = [
    "anthropic.claude-3-haiku-20240307",  # Claude 3 Haiku
    "amazon.titan-text-express-v1"        # Amazon Titan Text
]

comparison_results = compare_models(comparison_prompt, models_to_compare)

# Display results in markdown
for model, result in comparison_results.items():
    display(Markdown(f"## Response from {model}"))
    display(Markdown(result))
    display(Markdown("---"))

print("Note: Model comparison code is ready but commented out to avoid unnecessary API calls.")
print("During your actual demo, you can uncomment the code above to run a live comparison.")


## 3. Using System Prompts

System prompts help guide the model's behavior. Let's see how to use them:


In [ ]:
def demonstrate_system_prompt(model_id="anthropic.claude-3-haiku-20240307"):
    """
    Demonstrates how to use system prompts with Claude models
    """
    try:
        system_prompt = "You are a helpful AI assistant specialized in AWS services. Keep your answers concise and technical."
        user_prompt = "What is Amazon Bedrock and why would I use it?"
        
        print(f"System prompt: '{system_prompt}'")
        print(f"User prompt: '{user_prompt}'")
        
        request_body = {
            "anthropic_version": "bedrock-2023-05-31",
            "max_tokens": 500,
            "temperature": 0.7,
            "messages": [
                {
                    "role": "system",
                    "content": system_prompt
                },
                {
                    "role": "user",
                    "content": user_prompt
                }
            ]
        }
        
        response = bedrock_runtime.invoke_model(
            modelId=model_id,
            body=json.dumps(request_body)
        )
        
        response_body = json.loads(response.get('body').read())
        return response_body["content"][0]["text"]
        
    except ClientError as e:
        print(f"Error invoking model with system prompt: {e}")
        return None

# Run the system prompt demo
system_response = demonstrate_system_prompt()
display(Markdown(f"## Response with System Prompt"))
display(Markdown(system_response))

## 4. Streaming Responses

For more responsive applications, you can stream responses:

In [ ]:
def demonstrate_streaming(model_id="anthropic.claude-3-haiku-20240307", prompt="Tell me about AWS Bedrock in 3 sentences"):
    """
    Demonstrates streaming output from a Bedrock model
    """
    try:
        print(f"Sending streaming request to {model_id} with prompt: '{prompt}'")
        
        request_body = {
            "anthropic_version": "bedrock-2023-05-31",
            "max_tokens": 300,
            "temperature": 0.7,
            "messages": [
                {
                    "role": "user",
                    "content": prompt
                }
            ]
        }
        
        response = bedrock_runtime.invoke_model_with_response_stream(
            modelId=model_id,
            body=json.dumps(request_body)
        )
        
        stream = response.get('body')
        
        # In a notebook, we'll collect the chunks and then display them
        # In a real application, you would process these chunks as they arrive
        print("Receiving streaming response (chunks collected and displayed at the end):")
        chunks = []
        for event in stream:
            chunk = event.get('chunk')
            if chunk:
                chunk_data = json.loads(chunk.get('bytes').decode())
                if "content" in chunk_data and chunk_data["content"]:
                    content = chunk_data["content"][0]["text"]
                    chunks.append(content)
                    # For a real streaming demo, you could use this instead:
                    # display(content, display_id='streaming_output', update=True)
        
        return "".join(chunks)
        
    except ClientError as e:
        print(f"Error with streaming: {e}")
        return None

# Run the streaming demo
streaming_response = demonstrate_streaming()
display(Markdown(f"## Complete Streaming Response"))
display(Markdown(streaming_response))

## 5. Practical Example: Creating Documentation

Let's put everything together in a practical example where we ask the model to create documentation for an AWS service:


In [ ]:
def create_documentation(service_name, model_id="anthropic.claude-3-haiku-20240307"):
    """
    Use Bedrock to generate documentation for an AWS service
    """
    system_prompt = """
    You are an AWS documentation expert. Create concise, accurate, and helpful documentation.
    Format your response with markdown, including:
    - A brief service overview
    - Key features (as a bulleted list)
    - A simple code example
    - Common use cases
    """
    
    user_prompt = f"Create documentation for {service_name}."
    
    request_body = {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 1000,
        "temperature": 0.5,
        "messages": [
            {
                "role": "system",
                "content": system_prompt
            },
            {
                "role": "user",
                "content": user_prompt
            }
        ]
    }
    
    response = bedrock_runtime.invoke_model(
        modelId=model_id,
        body=json.dumps(request_body)
    )
    
    response_body = json.loads(response.get('body').read())
    return response_body["content"][0]["text"]

bedrock_docs = create_documentation("Amazon Bedrock")
display(Markdown(f"# Generated Documentation"))
display(Markdown(bedrock_docs))


## Conclusion

This notebook has demonstrated several ways to interact with Amazon Bedrock:

1. Basic model invocation
2. Comparing responses from different models
3. Using system prompts to guide model behavior
4. Streaming responses for more responsive applications
5. A practical example of generating documentation

These patterns form the foundation for building more complex GenAI applications on AWS.
